In [1]:
# =============================================================================
# CONFIGURACIÓN COMPLETA DE PREPROCESAMIENTO PARA EXPERIMENTOS
# =============================================================================

import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Descargar recursos de NLTK (ejecutar solo una vez si es necesario)
# nltk.download('stopwords')

# Inicializar stemmer y stopwords
stemmer = SnowballStemmer('spanish')
stop_words_es = set(stopwords.words('spanish'))
stop_words_en = set(stopwords.words('english'))

# Constantes para normalización
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS = "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def only_normalizar_texto(input_str, punct=False, accents=False, num=False, max_dup=2, lowercase=True):
    """
    Normalización avanzada de texto con control granular de parámetros
    
    Args:
        input_str: Texto a normalizar
        punct: False elimina puntuación, True la mantiene
        accents: False elimina acentos, True los mantiene  
        num: False elimina números, True los mantiene
        max_dup: Número máximo de caracteres duplicados consecutivos
        lowercase: True convierte a minúsculas
    
    Returns:
        Texto normalizado
    """
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    
    if lowercase:
        texto = texto.lower()
    
    return texto

def normalizar_txt_sin_StopWords(texto):
    """
    Normaliza el texto (eliminando puntuación, acentos y números) y elimina stopwords
    """
    # Usar función de normalización con configuración estándar para NLP
    texto_normalizado = only_normalizar_texto(
        texto, 
        punct=False,    # Eliminar puntuación
        accents=False,  # Eliminar acentos  
        num=False,      # Eliminar números
        max_dup=2,      # Máximo 2 duplicados
        lowercase=True  # Convertir a minúsculas
    )
    
    if not texto_normalizado:
        return ""
    
    # Tokenizar y eliminar stopwords (español e inglés)
    palabras = texto_normalizado.split()
    palabras_filtradas = [palabra for palabra in palabras 
                         if palabra not in stop_words_es and palabra not in stop_words_en]
    
    # Reconstruir el texto
    return ' '.join(palabras_filtradas)

def normalizar_txt_sin_StopWords_mas_stemming(texto):
    """
    Normaliza, elimina stopwords y aplica stemming
    """
    # Primero normalizar y eliminar stopwords
    texto_sin_stopwords = normalizar_txt_sin_StopWords(texto)
    
    if not texto_sin_stopwords:
        return ""
    
    # Aplicar stemming a cada palabra
    palabras = texto_sin_stopwords.split()
    palabras_stemmed = [stemmer.stem(palabra) for palabra in palabras]
    
    # Reconstruir el texto
    return ' '.join(palabras_stemmed)

def mi_preprocesamiento(tipo_procesamiento):
    """
    Función dispatcher que retorna la función de preprocesamiento especificada
    
    Args:
        tipo_procesamiento: 
            'only_normalizar_texto' - Solo normalización básica
            'normalizar_txt_sin_StopWords' - Normalización + sin stopwords  
            'normalizar_txt_sin_StopWords_mas_stemming' - Normalización + sin stopwords + stemming
    
    Returns:
        Función de preprocesamiento correspondiente
    """
    procesadores = {
        'only_normalizar_texto': lambda x: only_normalizar_texto(
            x, punct=False, accents=False, num=False, max_dup=2, lowercase=True
        ),
        'normalizar_txt_sin_StopWords': normalizar_txt_sin_StopWords,
        'normalizar_txt_sin_StopWords_mas_stemming': normalizar_txt_sin_StopWords_mas_stemming
    }
    
    if tipo_procesamiento not in procesadores:
        raise ValueError(f"Tipo de procesamiento no válido. Opciones: {list(procesadores.keys())}")
    
    return procesadores[tipo_procesamiento]

# =============================================================================
# EJEMPLOS DE USO Y PRUEBAS
# =============================================================================

def probar_preprocesamiento():
    """Función para probar todos los tipos de preprocesamiento"""
    
    texto_ejemplo = "¡¡¡ESTE es un TEXTO de ejemplo!!! con números 123 y acentos: áéíóú"
    
    print("=" * 70)
    print("PRUEBA DE PREPROCESAMIENTO")
    print("=" * 70)
    print(f"Texto original: {texto_ejemplo}")
    print("-" * 70)
    
    # Probar cada tipo de procesamiento
    tipos = [
        'only_normalizar_texto',
        'normalizar_txt_sin_StopWords', 
        'normalizar_txt_sin_StopWords_mas_stemming'
    ]
    
    for tipo in tipos:
        resultado = mi_preprocesamiento(tipo)(texto_ejemplo)
        print(f"{tipo}:")
        print(f"  → {resultado}")
        print()

def crear_vectorizadores_ejemplo():
    """Ejemplo de cómo crear vectorizadores con diferentes configuraciones"""
    
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    
    print("=" * 70)
    print("EJEMPLOS DE VECTORIZADORES")
    print("=" * 70)
    
    # Configuración 1: CountVectorizer con normalización básica y unigramas
    vec1 = CountVectorizer(
        analyzer="word",
        preprocessor=mi_preprocesamiento('only_normalizar_texto'),
        ngram_range=(1,1)
    ) 
    print("✓ CountVectorizer creado: normalización básica + unigramas")
    
    # Configuración 2: TF-IDF sin stopwords y unigramas+bigramas
    vec2 = TfidfVectorizer(
        analyzer="word",
        preprocessor=mi_preprocesamiento('normalizar_txt_sin_StopWords'),
        ngram_range=(1,2)
    )
    print("✓ TfidfVectorizer creado: sin stopwords + uni+bigramas")
    
    # Configuración 3: CountVectorizer con stemming
    vec3 = CountVectorizer(
        analyzer="word", 
        preprocessor=mi_preprocesamiento('normalizar_txt_sin_StopWords_mas_stemming'),
        ngram_range=(1,1)
    )
    print("✓ CountVectorizer creado: con stemming + unigramas")
    
    return vec1, vec2, vec3

# =============================================================================
# EJECUCIÓN DE PRUEBAS
# =============================================================================

if __name__ == "__main__":
    # Probar el preprocesamiento
    probar_preprocesamiento()
    
    # Crear ejemplos de vectorizadores
    vec1, vec2, vec3 = crear_vectorizadores_ejemplo()
    
    print("\n" + "=" * 70)
    print("¡CONFIGURACIÓN LISTA PARA EXPERIMENTOS!")
    print("=" * 70)
    print("\nPuedes usar:")
    print("  preprocessor=mi_preprocesamiento('tipo_procesamiento')")
    print("\nTipos disponibles:")
    print("  - only_normalizar_texto")
    print("  - normalizar_txt_sin_StopWords") 
    print("  - normalizar_txt_sin_StopWords_mas_stemming")

PRUEBA DE PREPROCESAMIENTO
Texto original: ¡¡¡ESTE es un TEXTO de ejemplo!!! con números 123 y acentos: áéíóú
----------------------------------------------------------------------
only_normalizar_texto:
  → este es un texto de ejemplo con numeros y acentos aeiou

normalizar_txt_sin_StopWords:
  → texto ejemplo numeros acentos aeiou

normalizar_txt_sin_StopWords_mas_stemming:
  → text ejempl numer acent aeiou

EJEMPLOS DE VECTORIZADORES
✓ CountVectorizer creado: normalización básica + unigramas
✓ TfidfVectorizer creado: sin stopwords + uni+bigramas
✓ CountVectorizer creado: con stemming + unigramas

¡CONFIGURACIÓN LISTA PARA EXPERIMENTOS!

Puedes usar:
  preprocessor=mi_preprocesamiento('tipo_procesamiento')

Tipos disponibles:
  - only_normalizar_texto
  - normalizar_txt_sin_StopWords
  - normalizar_txt_sin_StopWords_mas_stemming


In [ ]:
# Ejemplo directo en tu código
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizador con normalización básica
vec_basico = TfidfVectorizer(
    preprocessor=mi_preprocesamiento('only_normalizar_texto'),
    ngram_range=(1,1)
)

# Vectorizador sin stopwords  
vec_sin_stopwords = TfidfVectorizer(
    preprocessor=mi_preprocesamiento('normalizar_txt_sin_StopWords'),
    ngram_range=(1,2)
)

# Vectorizador con stemming
vec_stemming = TfidfVectorizer(
    preprocessor=mi_preprocesamiento('normalizar_txt_sin_StopWords_mas_stemming'),
    ngram_range=(1,1)
)